# vector stores and retriver 

vector stores 
retrviers do

In [1]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [2]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api=os.getenv("GROK_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

In [11]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api,model="llama-3.1-8b-instant")

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [14]:
#vector store
from langchain_chroma import Chroma
vectostore=Chroma.from_documents(documents=documents,embedding=embeddings)


In [15]:
vectostore.similarity_search("cat")

[Document(id='b505d409-de6c-4eb3-a62e-5d3db4149456', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='26f0523d-fe75-440a-a152-86452fe99b77', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='63f2bd29-e0d1-4d7e-895f-7428a5f40ae0', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='424f694d-f542-47df-83bb-5141a594f30f', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [16]:
# async query
await vectostore.asimilarity_search("cats")

[Document(id='b505d409-de6c-4eb3-a62e-5d3db4149456', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='63f2bd29-e0d1-4d7e-895f-7428a5f40ae0', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='26f0523d-fe75-440a-a152-86452fe99b77', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='424f694d-f542-47df-83bb-5141a594f30f', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [17]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriver=RunnableLambda(vectostore.similarity_search).bind(k=1)
retriver.batch(["cat","dog"])

[[Document(id='b505d409-de6c-4eb3-a62e-5d3db4149456', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='26f0523d-fe75-440a-a152-86452fe99b77', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

### vector store implment as the riterver methos

In [ ]:
retriver=vectostore.as_retriever(search_type="similarity",search_kwargs={"k":1
                                                                }
)

In [22]:
retriver.batch(["cat"])

[[Document(id='b505d409-de6c-4eb3-a62e-5d3db4149456', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]]

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

messages=""""
Answer this question by giving the provided context only.
{question}

Context:
{context}"""

In [27]:
prompt = ChatPromptTemplate.from_messages([
    ("human", messages)
])


rag_chain={"context":retriver,"question":RunnablePassthrough()}|prompt|llm

In [ ]:
response=rag_chain.invoke("tell about the lions")
print(response.content)

There is no information about lions in the provided context. The context only talks about dogs.


In [29]:
response=rag_chain.invoke("tell about the dogs")
print(response.content)

Dogs are great companions, known for their loyalty and friendliness.
